# Comment extraire du contenu textuel de manière ciblée ?

## Méthode simple et rapide (1) : Expressions régulières

In [1]:
import re

doc = '<html><head/><body><article><p>Paragraphe 1</p><p>Paragraphe 2</p></article></body></html>'
match = re.search(r'(?:<article>).+?(?:</article>)', doc)
print(match.group(0))
print()
cleaned = re.sub(r'<.+?>', '', doc)
print(cleaned)

<article><p>Paragraphe 1</p><p>Paragraphe 2</p></article>

Paragraphe 1Paragraphe 2


→ Problème : normalisation des espaces.

## Méthode simple (2) : Élements de structure

### Prérequis

Parseur : [LXML](https://lxml.de/)

Doit être installé si ce n'est pas déjà le cas : `pip install lxml`

In [2]:
from lxml import html
doc = '<html><head/><body><p>Paragraphe 1</p><p>Paragraphe 2</p></body></html>'
parsed = html.fromstring(doc)
for paragraph in parsed.iter('p'):
    print(paragraph.text)

Paragraphe 1
Paragraphe 2


→ Approche plus "propre" que les expressions régulières, les étiquettes HTML permettent une approche plus fine.

### Texte orphelin et autres tags invisibles

In [3]:
doc = '<html><head/><body><h1>Titre</h1><p>Paragraphe 1</p>Texte orphelin<p>Paragraphe 2</p></body></html>'
parsed = html.fromstring(doc)
for paragraph in parsed.iter('h1', 'p'):
    print(paragraph.text)

Titre
Paragraphe 1
Paragraphe 2


→ Résultat conforme mais problème : texte manquant ("Texte orphelin").

## Méthode déjà moins simple (3) : XPath

### XPath : Titres et paragraphes

In [4]:
from lxml import html

# lecture d'un fichier HTML
fichier = 'toto/antarctique-manque-criant-de-financements-pour-la-recherche-polaire-francaise-20210614.html'
with open(fichier, encoding='utf-8') as f:
    document = f.read()
# chargement dans LXML
arbre = html.fromstring(document)
# XPath
i = 0
for element in arbre.xpath('//h1|//p'):
    print(element.tag, '-', element.text_content())
    i += 1
    if i == 10:
        break

p - Rubriques et services du Figaro
p - 
p - Rubriques et services du Figaro
p - Nos journaux et magazines
p - Les sites du Groupe Figaro
h1 - Antarctique: manque criant de financements pour la recherche polaire française
p - Alors que la réunion internationale de l'Antarctique se tient cette année à Paris, et malgré ses deux bases permanentes sur le continent blanc, la France consacre moins d'argent pour la science polaire que des pays comme la Pologne, la Corée du Sud ou l'Australie.
p - «On est encore dans la cour des grands pour les résultats de recherche. Mais depuis 2005, les moyens financiers pour la recherche polaire sont rognés peu à peu. Ils sont tombés à moins de 20 millions d'euros par an. Cette enveloppe est indigne d'un grand pays comme la France », se désole Yvon Le Maho, président du conseil d'administration de l'IPEV (Institut Polaire français Paul-Émile Victor), biologiste de réputation internationale et membre de l'Académie des sciences.
p - Pour l'instant, la France

→ Les éléments choisis ciblent plus efficacement le contenu mais ce n'est pas parfait.

### XPath : Article

In [5]:
for element in arbre.xpath('//article'):
    print(element.text_content()[:500])
    i += 1
    if i == 1:
        break

    
        Réservé aux abonnés
    
Antarctique: manque criant de financements pour la recherche polaire française
Alors que la réunion internationale de l'Antarctique se tient cette année à Paris, et malgré ses deux bases permanentes sur le continent blanc, la France consacre moins d'argent pour la science polaire que des pays comme la Pologne, la Corée du Sud ou l'Australie.

                Par Marc Cherki
                Publié 
                    hier à 17:45, Mis à jour hier à 18:35
   


→ Article trouvé mais la mise en forme peut poser problème.

### XPath : Article + Trimming

In [6]:
from trafilatura.utils import sanitize

element = arbre.xpath('//article')[0]
print(sanitize(element.text_content()))

Réservé aux abonnés
Antarctique: manque criant de financements pour la recherche polaire française
Alors que la réunion internationale de l'Antarctique se tient cette année à Paris, et malgré ses deux bases permanentes sur le continent blanc, la France consacre moins d'argent pour la science polaire que des pays comme la Pologne, la Corée du Sud ou l'Australie.
ParMarc Cherki
Publié
hier à 17:45, Mis à jour hier à 18:35
La base française de Dumont d'Urville, en Antarctique, attend depuis 2005 sa rénovation (photo prise le 15 février 2021). Crédits photo. Alamy/ABACA
«On est encore dans la cour des grands pour les résultats de recherche. Mais depuis 2005, les moyens financiers pour la recherche polaire sont rognés peu à peu. Ils sont tombés à moins de 20 millions d'euros par an. Cette enveloppe est indigne d'un grand pays comme la France», se désole Yvon Le Maho, président du conseil d'administration de l'IPEV (Institut Polaire français Paul-Émile Victor), biologiste de réputation inter

→ Article trouvé et nettoyé (éléments de structure perdus au passage).

### XPath : Article + Détourage par attributs (→ premium)

In [7]:
from copy import copy

# il faut copier l'objet sans quoi les modifications se répercutent sur l'original
arbre_test = copy(arbre)

for element in arbre_test.xpath('//*[contains(@class, "premium")]'):
    element.getparent().remove(element)

print(len(arbre_test.xpath('//article')))

0


→ Arbre vide... Trop de texte enlevé !

### XPath : Article + Détourage des divs premium

In [8]:
arbre_test = copy(arbre)

for element in arbre_test.xpath('//div[contains(@class, "premium")]'):
    element.getparent().remove(element)

element = arbre_test.xpath('//article')[0]
print(sanitize(element.text_content()))

Antarctique: manque criant de financements pour la recherche polaire française
Alors que la réunion internationale de l'Antarctique se tient cette année à Paris, et malgré ses deux bases permanentes sur le continent blanc, la France consacre moins d'argent pour la science polaire que des pays comme la Pologne, la Corée du Sud ou l'Australie.
ParMarc Cherki
Publié
hier à 17:45, Mis à jour hier à 18:35
La base française de Dumont d'Urville, en Antarctique, attend depuis 2005 sa rénovation (photo prise le 15 février 2021). Crédits photo. Alamy/ABACA
«On est encore dans la cour des grands pour les résultats de recherche. Mais depuis 2005, les moyens financiers pour la recherche polaire sont rognés peu à peu. Ils sont tombés à moins de 20 millions d'euros par an. Cette enveloppe est indigne d'un grand pays comme la France», se désole Yvon Le Maho, président du conseil d'administration de l'IPEV (Institut Polaire français Paul-Émile Victor), biologiste de réputation internationale et membre 

→ Résultat déjà bien plus probant !

Souci annexe mais pas futile : l'article est payant...

## Conclusions

- Le détourage au cas par cas est possible mais délicat !
- La complexité et la variabilité des documents HTML pose problème :
   - Extraire du texte oui mais quoi ? Quelles sections du document ?
   - Quelles étapes de post-processing ? (nettoyage et normalisation)
- Un outil générique permet de gagner du temps, avec une perte en terme de précision ou de rappel qu'il convient d'observer.